## Build a LLM technical tutor that answers with examples and prompts for a quiz

End of week 1 exercise (LLM Engineer course on Udemy by Ed Donner)

Reference: https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
def get_tutor_user_prompt(question):
    user_prompt = question
    
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
system_prompt = "You are an expert tutor that explains with examples and also generates quiz (if the student agrees)  \
"

In [ ]:
def stream_answer(llm_api, MODEL, question):
    stream = llm_api.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_tutor_user_prompt(question)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()
stream_answer(openai, MODEL_GPT,question)


In [ ]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
stream_answer(ollama_via_openai, MODEL_LLAMA, question)
